In [1]:
#Base

import pandas as pd
import os

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

from sklearn.feature_selection import RFE

import statsmodels.api as sm

from tqdm import tqdm


dataset_folder = os.path.join(os.getcwd(), 'dataset')

def df_from_csv(filename):
    f = os.path.join(dataset_folder, filename)
    return pd.read_csv(f, delimiter='\t|\n|,', engine='python')

x_train = df_from_csv('ori_trainx.csv')
y_train = df_from_csv('ori_trainy.csv')
alert_date = df_from_csv('ori_alert_date.csv')
alert_cust = df_from_csv('ori_custinfo.csv')
xdp = df_from_csv('ori_xdp.csv')

smote = SMOTE(random_state=0)
tqdm.pandas(desc='Progress Bar')

def SMOTE_oversampling(input_x, input_y):
    x_train, x_test, y_train, y_test = train_test_split(input_x, input_y, test_size=0.3)
    x_cols = x_train.columns
    y_cols = y_train.columns

    sm_data_x, sm_data_y = smote.fit_resample(x_train, y_train)
    sm_data_x = pd.DataFrame(data=sm_data_x, columns=x_cols)
    sm_data_y = pd.DataFrame(data=sm_data_y, columns=y_cols)

    print("length of oversampled data is ",len(sm_data_x))
    print("Number of non-sar in oversampled data",len(sm_data_y[sm_data_y['sar_flag']==0]))
    print("Number of sar",len(sm_data_y[sm_data_y['sar_flag']==1]))
    print("Proportion of no subscription data in oversampled data is ", len(sm_data_y[sm_data_y['sar_flag']==0])/len(sm_data_x))
    print("Proportion of subscription data in oversampled data is ", len(sm_data_y[sm_data_y['sar_flag']==1])/len(sm_data_x))

    return sm_data_x, sm_data_y

def RFE_analysis(input_x, input_y, model):
    rfe = RFE(model, step=300)
    rfe.fit(input_x, input_y.values.ravel())
    print(input_x.columns)
    print(rfe.support_)
    print(rfe.ranking_)
    return rfe.get_support(1)

def LR_analysis(input_x, input_y):
    sm_LR = sm.Logit(input_y, input_x)
    result = sm_LR.fit()
    print(result.summary2())

def LR_training(input_x, input_y, LR_model):
    x_train, x_test, y_train, y_test = train_test_split(input_x, input_y, test_size=0.25)
    LR_model.fit(x_train, y_train)
    y_train_predict = LR_model.predict(x_train)
    y_test_predict = LR_model.predict(x_test)
    train_acc = accuracy_score(y_train, y_train_predict)
    test_acc = accuracy_score(y_test, y_test_predict)
    print(f'{train_acc}  {test_acc}')
    cm = confusion_matrix(y_true=y_test, y_pred=y_test_predict)
    print(cm)
    return LR_model

def LR_test(input_x, input_y, LR_model):
    y_predict = LR_model.predict(input_x)
    test_acc = accuracy_score(input_y, y_predict)
    print(f'{test_acc}')
    cm = confusion_matrix(y_true=input_y, y_pred=y_predict)
    print(cm)

def calculate_tx(row):
    if not pd.isna(row['risk_rank']):
        temp_frame = xdp[(xdp['tx_date'] <= row['date']) & (xdp['cust_id'] == row['cust_id'])]
        total_tx = (temp_frame['tx_amt'] * temp_frame['exchg_rate'])
        row['total_tx'] = total_tx.sum()
        row['avg_tx'] = total_tx.mean()
        row['count_tx'] = total_tx.count()
        row['std_tx'] = total_tx.std()
        row['tx_per_day'] = row['count_tx']/row['date'] if row['date'] > 0 else 0
    return row


C:\Software\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Software\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
# Join data for training

ccba = pd.read_pickle('aggregated_ccba_cust.pkl')
cdtx = pd.read_pickle('cdtx_agg.pkl')
xdp = pd.read_pickle('xdp_train_test1.pkl')

In [19]:
train = df_from_csv('ori_trainx.csv')
train = train.merge(df_from_csv('ori_trainy.csv'), on='alert_key', how='inner')
train = train.merge(alert_cust, on='alert_key', how='left')
train = train[['alert_key', 'cust_id']]
train = train.merge(ccba, on='cust_id', how='left')
train = train.merge(cdtx, on='cust_id', how='left')
train = train.merge(xdp, on='alert_key', how='left')
train = train.reindex(sorted(train.columns), axis=1)
train

,AGE,alert_key,avg_amt_nontw_nonntd,avg_amt_nontw_ntd,avg_amt_tw_nonntd,avg_amt_tw_ntd,avg_freq_nontw_nonntd,avg_freq_nontw_ntd,avg_freq_tw_nonntd,avg_freq_tw_ntd,...,total_amt_nontw_nonntd,total_amt_nontw_ntd,total_amt_tw_nonntd,total_amt_tw_ntd,total_freq_nontw_nonntd,total_freq_nontw_ntd,total_freq_tw_nonntd,total_freq_tw_ntd,total_tx,tx_per_day
0,4,171189,9044.740741,3033.355556,NaN,1388.550403,1.350000,1.097561,NaN,2.147186,...,244208.0,136501.0,NaN,688721.0,27.0,45.0,NaN,496.0,1.515000e+03,0.000000
1,2,171202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.092020e+05,0.000000
2,4,171599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.105930e+05,0.000000
3,4,171737,NaN,NaN,NaN,1893.333333,NaN,NaN,NaN,1.500000,...,NaN,NaN,NaN,5680.0,NaN,NaN,NaN,3.0,4.312500e+04,0.000000
4,3,171142,NaN,NaN,NaN,7048.368421,NaN,NaN,NaN,1.055556,...,NaN,NaN,NaN,133919.0,NaN,NaN,NaN,19.0,2.963440e+05,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23901,3,352132,4332.523077,1223.939446,NaN,988.503826,1.160714,2.513043,NaN,6.668367,...,281614.0,707437.0,NaN,2583949.0,65.0,578.0,NaN,2614.0,1.532000e+08,0.390110
23902,3,352125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.615283e+07,0.054945
23903,6,352080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.555772e+08,0.192308
23904,6,352075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.365559e+08,1.181319


In [20]:
train.iloc[:, 2:] = train.iloc[:, 2:].fillna(0)
train = train.drop(['cust_id_x', 'cust_id_y'], axis=1)
train.to_pickle('aggregate.pkl')

In [21]:
train = train.drop(['alert_key'], axis=1)
train

,AGE,avg_amt_nontw_nonntd,avg_amt_nontw_ntd,avg_amt_tw_nonntd,avg_amt_tw_ntd,avg_freq_nontw_nonntd,avg_freq_nontw_ntd,avg_freq_tw_nonntd,avg_freq_tw_ntd,avg_tx,...,total_amt_nontw_nonntd,total_amt_nontw_ntd,total_amt_tw_nonntd,total_amt_tw_ntd,total_freq_nontw_nonntd,total_freq_nontw_ntd,total_freq_tw_nonntd,total_freq_tw_ntd,total_tx,tx_per_day
0,4,9044.740741,3033.355556,0.0,1388.550403,1.350000,1.097561,0.0,2.147186,5.050000e+02,...,244208.0,136501.0,0.0,688721.0,27.0,45.0,0.0,496.0,1.515000e+03,0.000000
1,2,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,6.973400e+04,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.092020e+05,0.000000
2,4,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,7.764825e+04,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.105930e+05,0.000000
3,4,0.000000,0.000000,0.0,1893.333333,0.000000,0.000000,0.0,1.500000,4.312500e+03,...,0.0,0.0,0.0,5680.0,0.0,0.0,0.0,3.0,4.312500e+04,0.000000
4,3,0.000000,0.000000,0.0,7048.368421,0.000000,0.000000,0.0,1.055556,4.939067e+03,...,0.0,0.0,0.0,133919.0,0.0,0.0,0.0,19.0,2.963440e+05,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23901,3,4332.523077,1223.939446,0.0,988.503826,1.160714,2.513043,0.0,6.668367,1.078873e+06,...,281614.0,707437.0,0.0,2583949.0,65.0,578.0,0.0,2614.0,1.532000e+08,0.390110
23902,3,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,2.307641e+06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.615283e+07,0.054945
23903,6,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,2.222531e+06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.555772e+08,0.192308
23904,6,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,3.175717e+05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.365559e+08,1.181319


In [6]:
# Training
train_x = train.loc[:, train.columns != 'sar_flag']
train_y = train.loc[:, train.columns == 'sar_flag']
train_x

,AGE,avg_amt_nontw_nonntd,avg_amt_nontw_ntd,avg_amt_tw_nonntd,avg_amt_tw_ntd,avg_freq_nontw_nonntd,avg_freq_nontw_ntd,avg_freq_tw_nonntd,avg_freq_tw_ntd,avg_tx,...,total_amt_nontw_nonntd,total_amt_nontw_ntd,total_amt_tw_nonntd,total_amt_tw_ntd,total_freq_nontw_nonntd,total_freq_nontw_ntd,total_freq_tw_nonntd,total_freq_tw_ntd,total_tx,tx_per_day
0,4,9044.740741,3033.355556,0.0,1388.550403,1.350000,1.097561,0.0,2.147186,5.050000e+02,...,244208.0,136501.0,0.0,688721.0,27.0,45.0,0.0,496.0,1.515000e+03,0.000000
1,2,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,6.973400e+04,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.092020e+05,0.000000
2,4,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,7.764825e+04,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.105930e+05,0.000000
3,4,0.000000,0.000000,0.0,1893.333333,0.000000,0.000000,0.0,1.500000,4.312500e+03,...,0.0,0.0,0.0,5680.0,0.0,0.0,0.0,3.0,4.312500e+04,0.000000
4,3,0.000000,0.000000,0.0,7048.368421,0.000000,0.000000,0.0,1.055556,4.939067e+03,...,0.0,0.0,0.0,133919.0,0.0,0.0,0.0,19.0,2.963440e+05,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23901,3,4332.523077,1223.939446,0.0,988.503826,1.160714,2.513043,0.0,6.668367,1.078873e+06,...,281614.0,707437.0,0.0,2583949.0,65.0,578.0,0.0,2614.0,1.532000e+08,0.390110
23902,3,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,2.307641e+06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.615283e+07,0.054945
23903,6,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,2.222531e+06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.555772e+08,0.192308
23904,6,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,3.175717e+05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.365559e+08,1.181319


In [7]:
sm_x, sm_y = SMOTE_oversampling(train_x, train_y)

length of oversampled data is  33116
Number of non-sar in oversampled data 16558
Number of sar 16558
Proportion of no subscription data in oversampled data is  0.5
Proportion of subscription data in oversampled data is  0.5


In [8]:
LR = LogisticRegression(max_iter=400, n_jobs=-1)
rfe_columns = RFE_analysis(sm_x, sm_y, LR)

Index(['AGE', 'avg_amt_nontw_nonntd', 'avg_amt_nontw_ntd', 'avg_amt_tw_nonntd',
       'avg_amt_tw_ntd', 'avg_freq_nontw_nonntd', 'avg_freq_nontw_ntd',
       'avg_freq_tw_nonntd', 'avg_freq_tw_ntd', 'avg_tx', 'count_tx',
       'cucah_quarter1_mean', 'cucah_quarter1_std', 'cucah_quarter1_sum',
       'cucah_quarter2_mean', 'cucah_quarter2_std', 'cucah_quarter2_sum',
       'cucah_quarter3_mean', 'cucah_quarter3_std', 'cucah_quarter3_sum',
       'cucah_quarter4_mean', 'cucah_quarter4_std', 'cucah_quarter4_sum',
       'cucah_total_mean', 'cucah_total_std', 'cucah_total_sum', 'date',
       'risk_rank', 'sd_amt_nontw_nonntd', 'sd_amt_nontw_ntd',
       'sd_amt_tw_nonntd', 'sd_amt_tw_ntd', 'sd_freq_nontw_nonntd',
       'sd_freq_nontw_ntd', 'sd_freq_tw_nonntd', 'sd_freq_tw_ntd', 'std_tx',
       'total_amt_nontw_nonntd', 'total_amt_nontw_ntd', 'total_amt_tw_nonntd',
       'total_amt_tw_ntd', 'total_freq_nontw_nonntd', 'total_freq_nontw_ntd',
       'total_freq_tw_nonntd', 'total_freq_t

In [9]:
sm_x = sm_x[sm_x.columns[rfe_columns]]
sm_x

,avg_amt_nontw_nonntd,avg_amt_nontw_ntd,avg_amt_tw_ntd,avg_tx,cucah_quarter1_mean,cucah_quarter1_std,cucah_quarter1_sum,cucah_quarter2_mean,cucah_quarter2_sum,cucah_quarter3_mean,...,cucah_total_mean,cucah_total_std,cucah_total_sum,date,sd_amt_nontw_nonntd,sd_amt_nontw_ntd,sd_amt_tw_ntd,total_amt_nontw_ntd,total_freq_nontw_nonntd,total_freq_tw_ntd
0,0.000000,0.000000,0.000000,52585.666667,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,194,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,254956.051650,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,116,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,120.615385,849.085427,138016.639144,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,354,0.000000,251.813270,2238.120596,1568.000000,0.000000,398.000000
4,0.000000,0.000000,0.000000,331541.172577,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,138,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33111,0.000000,0.000000,0.000000,34190.745211,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,318,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
33112,0.000000,0.000000,59178.312617,35255.202941,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,73,0.000000,0.000000,44380.687434,0.000000,0.000000,314.527356
33113,485062.939708,1661.814989,1710.098824,1192.102536,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,53,316457.159146,2998.266829,3747.259506,289155.808156,1.514423,404.528880
33114,1264.069639,1533.364449,717.058634,42382.335636,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,162,2074.989116,2870.437562,375.260767,709947.739873,12.352338,31.292589


In [10]:
# LR_analysis(sm_x, sm_y)

In [11]:
LR = LR_training(sm_x, sm_y, LR)

C:\Software\Anaconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.597696984337883  0.5923420702983452
[[3115  974]
 [2401 1789]]


In [13]:
# Training Evaluation
train_x_rfe = train_x[train_x.columns[rfe_columns]]
LR_test(train_x_rfe, train_y, LR)

# LR_test(train_x, train_y, LR)

0.7551660670961265
[[17958  5714]
 [  139    95]]


In [14]:
# Test
xdp_test = pd.read_pickle('submit_format.pkl')

test = df_from_csv('submit_format.csv')
test = test.drop(['probability'], axis=1)
test = test.merge(alert_cust, on='alert_key', how='left')
test = test[['alert_key', 'cust_id']]
test = test.merge(ccba, on='cust_id', how='left')
test = test.merge(cdtx, on='cust_id', how='left')
test = test.merge(xdp_test.drop(['occupation_code', 'total_asset'], axis=1), on='alert_key', how='left')
test.iloc[:, 2:] = test.iloc[:, 2:].fillna(0)
test = test.drop(['cust_id_x', 'cust_id_y', 'alert_key'], axis=1)
test = test.reindex(sorted(test.columns), axis=1)
test

,AGE,avg_amt_nontw_nonntd,avg_amt_nontw_ntd,avg_amt_tw_nonntd,avg_amt_tw_ntd,avg_freq_nontw_nonntd,avg_freq_nontw_ntd,avg_freq_tw_nonntd,avg_freq_tw_ntd,avg_tx,...,total_amt_nontw_nonntd,total_amt_nontw_ntd,total_amt_tw_nonntd,total_amt_tw_ntd,total_freq_nontw_nonntd,total_freq_nontw_ntd,total_freq_tw_nonntd,total_freq_tw_ntd,total_tx,tx_per_day
0,3.0,0.0,0.000000,0.0,78507.452830,0.0,0.000000,0.0,1.127660,3.715794e+04,...,0.0,0.0,0.0,4160895.0,0.0,0.0,0.0,53.0,1.040422e+07,0.756757
1,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000
2,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000
3,5.0,0.0,0.000000,0.0,1732.125000,0.0,0.000000,0.0,1.000000,3.953497e+05,...,0.0,0.0,0.0,13857.0,0.0,0.0,0.0,8.0,1.407445e+08,0.962162
4,3.0,0.0,154.000000,0.0,304.458937,0.0,1.000000,0.0,1.739496,1.391112e+06,...,0.0,308.0,0.0,63023.0,0.0,2.0,0.0,207.0,3.380401e+08,0.653226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3845,2.0,1115.5,361.000000,0.0,915.968647,2.0,1.030303,0.0,2.033557,2.116042e+04,...,2231.0,12274.0,0.0,555077.0,2.0,34.0,0.0,606.0,7.173381e+06,0.862595
3846,2.0,0.0,292.657895,0.0,1317.712871,0.0,1.027027,0.0,1.364865,1.021493e+06,...,0.0,11121.0,0.0,133089.0,0.0,38.0,0.0,101.0,5.413915e+08,1.352041
3847,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000
3848,3.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,2.265351e+06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.999934e+08,0.808901


In [15]:
y_prob = LR.predict_proba(test[test.columns[rfe_columns]])
# y_prob = LR.predict_proba(test)
y_prob

array([[0.98739898, 0.01260102],
       [0.49999996, 0.50000004],
       [0.49999996, 0.50000004],
       ...,
       [0.49999996, 0.50000004],
       [0.45957617, 0.54042383],
       [0.49999996, 0.50000004]])

# DNN

In [16]:
# DNN
# https://towardsdatascience.com/deep-learning-with-python-neural-networks-complete-tutorial-6b53c0b06af0
# https://towardsdatascience.com/deep-neural-networks-for-regression-problems-81321897ca33

from keras import models, layers, utils, backend as K
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

input_x = sm_x
input_y = sm_y
n_features = len(input_x.columns)
epoch = 50

inputs = layers.Input(name='input', shape=(n_features,))

h1 = layers.Dense(name='h1', units=int(round((n_features+1)/2)), activation='relu')(inputs)
h1 = layers.Dropout(name='drop1', rate=0.2)(h1)

h2 = layers.Dense(name='h2', units=int(round((n_features+1)/4)), activation='relu')(h1)
h2 = layers.Dropout(name='drop2', rate=0.2)(h2)

outputs = layers.Dense(name='output', units=1, activation='sigmoid')(h2)

DNN_1 = models.Model(inputs=inputs, outputs=outputs, name='DNN_1')
DNN_1.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
DNN_1.summary()

Model: "DNN_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 23)]              0         
                                                                 
 h1 (Dense)                  (None, 12)                288       
                                                                 
 drop1 (Dropout)             (None, 12)                0         
                                                                 
 h2 (Dense)                  (None, 6)                 78        
                                                                 
 drop2 (Dropout)             (None, 6)                 0         
                                                                 
 output (Dense)              (None, 1)                 7         
                                                                 
Total params: 373
Trainable params: 373
Non-trainable params:

In [17]:
# Save checkpoint in case learning stopped in the middle
# param 'mode' is based on 'monitor' param
# classification: monitor accuracy
# regression: monitor loss

checkpoint_path = "weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
callbacks = [checkpoint]

In [18]:
DNN_1.fit(x=input_x, y=input_y, batch_size=32, epochs=epoch, validation_split=0.3, callbacks=callbacks)

Epoch 1/50
712/725 [============================>.] - ETA: 0s - loss: 0.3003 - mean_absolute_error: 0.3003
Epoch 1: val_loss improved from inf to 1.00000, saving model to weights-improvement-01-1.00.hdf5
725/725 [==============================] - 1s 1ms/step - loss: 0.3000 - mean_absolute_error: 0.3000 - val_loss: 1.0000 - val_mean_absolute_error: 1.0000
Epoch 2/50
715/725 [============================>.] - ETA: 0s - loss: 0.2897 - mean_absolute_error: 0.2897
Epoch 2: val_loss did not improve from 1.00000
725/725 [==============================] - 1s 851us/step - loss: 0.2897 - mean_absolute_error: 0.2897 - val_loss: 1.0000 - val_mean_absolute_error: 1.0000
Epoch 3/50
654/725 [==========================>...] - ETA: 0s - loss: 0.2898 - mean_absolute_error: 0.2898
Epoch 3: val_loss did not improve from 1.00000
725/725 [==============================] - 1s 833us/step - loss: 0.2900 - mean_absolute_error: 0.2900 - val_loss: 1.0000 - val_mean_absolute_error: 1.0000
Epoch 4/50
724/725 [=====

KeyboardInterrupt: 

In [75]:
best_file = 'weights-improvement-01-1.00.hdf5'
DNN_1.load_weights(best_file)
DNN_1.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [76]:
DNN_y_pred = DNN_1.predict(input_x)

1035/1035 [==============================] - 1s 459us/step


In [77]:
DNN_y_pred

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)